In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_raw = pd.read_csv('/Users/r.malinin/Desktop/input_data.csv', sep = ';')

In [3]:
columns = ['house_id', 'object_type', 'id_region', 'price', 'level', 'levels', 'rooms', 'area',
             'kitchen_area', 'building_type']

# Оставляем нужны колонки для анализа:
df = df_raw[columns].copy()

In [4]:
# Очистим колонку 'house_id' от NaN и inf
df['house_id'] = df['house_id'].replace([float('nan'), float('inf')], None)

# Преобразуем id домов в формат integer:
df['house_id'] = df['house_id'].astype(int)

# Уберем дубликаты:
df = df.drop_duplicates()

In [5]:
# Заменяем - 100 на null в значениях площади кухни:
df['kitchen_area'] = np.where(df['kitchen_area'] == -100, np.nan, df['kitchen_area'])

# Заменяем - на +
df['kitchen_area'] = np.where(df['kitchen_area'] < 0, df['kitchen_area'] * (-1), df['kitchen_area'])

# Заменим нулевые цены на null:
df['price'] = np.where(df['price'] == 0, np.nan, df['price'])

# Заменим нулевые площади кухни на null:
df['kitchen_area'] = np.where(df['kitchen_area'] == 0, np.nan, df['kitchen_area'])

# где площадь кухни больше площади всего помещения, сделаем площадь кухни = null:
df['kitchen_area'] = np.where(df['area'] <=  df['kitchen_area'], np.nan, df['kitchen_area'])

In [6]:
# По данным на 2023 год самая дорогая недвижимость в России стоит 2.94 млрд рублей
# Тогдаб все что выше этой цуны - выбросы, их сделаем равными null:
the_richest_estate = 2 * 10**9
df['price'] = np.where(df['price'] > the_richest_estate, np.nan, df['price'])

# Также будем считать, что самая дорогая недвижимость - в Москве
# и если есть недвижисочть дороже самой дорогой недвижимости в Москве, то это тоже выброс:

the_richest_estate_msc = df[df['id_region'] == 77]['price'].max()

df['price'] = np.where(df['price'] > the_richest_estate_msc, np.nan, df['price'])

# Будем считать, что самая дешевая недвижимость стоит не менее 100 тыс руб (например, какая-нибудь небольшой домик)
# все, что ниже - выбросы, мы их обернем в null:

df['price'] = np.where(df['price'] < 10**5, np.nan, df['price'])

In [7]:
# Если этаж квартиры больше, чем этажей в доме, то значит, в колонке levels должно быть null:
df['levels'] = np.where(df['levels'] < df['level'], np.nan, df['levels'])

# Не бывает домов с 0 этажей, поэтому тоже будем считать, что там нет данных:
df['levels'] = np.where(df['levels'] == 0, np.nan, df['levels'])

df['level'] = np.where(df['level'] == 0, np.nan, df['level'])

In [8]:
# Загрузим справочник городов России:
df_regions = pd.read_excel('/Users/r.malinin/Desktop/regions.xlsx')

# Сджойним два датасета:
df = df.merge(df_regions, how = 'left', on = 'id_region')

In [9]:
# Не приджойнился регион с id_region = 200, региона с таким id нет в справочнике, 
# поэтому мы обзоем его "Неизвестно":
df['region'] = np.where(df['id_region'] == 200, 'Неизвестный регион', df['region'])

In [10]:
display(df.head(10))

,house_id,object_type,id_region,price,level,levels,rooms,area,kitchen_area,building_type,region
0,1632918,2,66,2451300.0,15.0,31.0,1,30.3,NaN,0,Свердловская область
1,1632918,0,1,1450000.0,5.0,5.0,1,33.0,6.0,0,Республика Адыгея (Адыгея)
2,681306,0,50,10700000.0,4.0,13.0,3,85.0,12.0,3,Московская область
3,681306,0,1,3100000.0,3.0,5.0,3,82.0,9.0,0,Республика Адыгея (Адыгея)
4,1730985,2,23,2500000.0,2.0,3.0,1,30.0,9.0,3,Краснодарский край
5,1009994,0,34,1450000.0,5.0,5.0,2,47.0,6.0,2,Волгоградская область
6,2823596,0,54,9000000.0,2.0,4.0,3,107.4,21.3,4,Новосибирская область
7,2823596,0,3,2990000.0,1.0,2.0,3,54.0,7.0,0,Республика Бурятия
8,1284243,0,23,2300000.0,16.0,18.0,1,39.7,11.5,4,Краснодарский край
9,1284243,0,58,2290000.0,2.0,2.0,2,53.2,16.0,5,Пензенская область


In [14]:
# Сохраним почищенный датафрейм для дальнейшей визуализации в Tableau:

file_path = '/Users/r.malinin/Desktop/LIFE/Магистратура/3 семетстр/russian_real_estate.csv' 

# Сохраните DataFrame в файл CSV
df.to_csv(file_path, index=False)  